<a href="https://colab.research.google.com/github/sv2639/proj3/blob/main/proj3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem 2 Implementing a Meet-in-the-Middle Attack on a Mini Block Cipher

### Project Overview:

This project involves students in the implementation and analysis of a meet-in-the-middle (MITM) attack against a simplified block cipher called mini block cipher based on the idea of the SAES (FYI, it is better to
use the simplified DES). The primary goal is to provide hands-on experience with the MITM attack, showcasing its effectiveness against certain cryptographic algorithms and understanding why modern ciphers like AES are designed to be immune to such attacks.

MITM or Meet-in-the-Middle Attack” is an exhaustive key search attack1. It is a cryptanalytic attack applicable to ciphers based on composition of multiple rounds of substitutions and permutations. It works by finding plaintext-ciphertext pairs that map to the same intermediate value after partial encryption/decryption2.

SAES starts with the key expansion, then works on encryption to get ciphertext and on decryption to recover the plaintext. The key expansion generates three keys. The first key, Key0, is used for the add round key to the plaintext. The second key, Key1, is used to perform Round 1 transformation on state, defined as encrypt_round1(). The third key, Key2, is used to perform Round 2 transformations on state,
defined as encrypt_round2(). For decryption, it is reverse. Key2 is used to perform inverse Round 2 transformations on ciphertext, defined as decrypt_round2(). Key1 is used to perform inverse Round 1
transformations on state, defined as decrypt_round1(). So, the pseudo-code for SAES would be the following. It is assumed the key size is 16 bit.

1. Get (Key0, Key1, Key2) from Key K using the key expansion.

2. Two rounds of Encrypt to get the ciphertext.
  
        I. AddRoundKey Key0
        
        II. encrypt_round1() using Key1
    
            i. Subsititute()
    
            ii. Shift()
    
            iii. Mix()
    
            iv. AddRoundKey()
        
        III. encrypt_round2() using Key2
    
            i. Subsititute()
    
            ii. Shift()
    
            iii. AddRoundKey()

3. Two rounds of decryption to recover the plaintext
  
        I. decrypt_round2() using Key 2
    
            i. AddRoundKey()
    
            ii. Shift()
    
            iii. Subsititute()
  
        II. decrypt_round1() using Key1

            i. AddRoundKey()
            
            ii. Mix()
            
            iii. Shift()
            
            iv. Subsititute()

        III. AddRoundKey Key0

For the project, we need to modify the pseudo code to fit our class project. The pseudo-code for this
mini block cipher based on the SAES is as follows:

1. Get (Key0, Key1, Key2) from Key K using the key expansion.

2. Two rounds of Encrypt to get the ciphertext.

        I. encrypt_round1() using Key1 on plaintext, P, and get intermediate state X

            i. Subsititute()

            ii. Shift()

            iii. Mix()

            iv. AddRoundKey()

        II. encrypt_round2() using Key2 on intermediate state X, and get the ciphertext C.

            i. Subsititute()
  
            ii. Shift()

            iii. AddRoundKey()

3. Two rounds of decryption to recover the plaintext

        I. decrypt_round2() using Key 2 on the ciphertext, C, and get intermediate state, Y

            i. AddRoundKey()

            ii. Shift()
  
            iii. Subsititute()

        II. decrypt_round1() using Key1 on Y to get plaintext P.

            i. AddRoundKey()

            ii. Mix()

            iii. Shift()

            iv. Subsititute()

The meet in the middle attack strategy to mini block cipher:

      A. Calculate X = encrypt_round1(Key1, P)

      B. Calculate X’ = decrypt_round2(Key2, C).

      C. Find out a pair (Key1, Key2) such at X = X’

      D. For one specific (P, C), there will be many matched pairs, we need to use another plaintext and ciphertext pair to eliminate some of the matched pairs. Ideally, we shall get one matched key pair. This pair can be used to get plaintext from any cyphertext. In other words, we cracked the block cipher.

Project Tasks

Task 1: Implementing Mini Block Cipher with key size 16 bit and block size 16 bit:

      a) Students will implement the Mini Block Cipher encryption and decryption functions (1, 2I, 2II, 3I, 3II) using jupyter notebook.

      b) Make at least ten pairs of plaintexts and ciphertexts.

Task 2: Meet-in-the-Middle Attack Implementation:

      a) Students need to implement the meet in the middle attack strategy to mini block cipher (a-d).

      b) Show key pair(s) that works for the pair of plaintext and ciphertext from task1 (b). Ideally, it should have only one key pair works.

Task 3: Analyze the time and memory complexity of the attack compared with the naive exhaustive key search.

      a) What is the key space for the mini block cipher?

      b) Image the mini block cipher is executed twice to generate a cipher text. It is called double mini cipher block. We need a key in 32 bits. The first 16 to the first mini block cipher, the remaining 16 to the second mini block cipher. The meet in the middle attack is to match the state for the first encryption of mini block cipher and the second decryption mini block. How many operations are needed to such attack?

      c) If we do exhaustive key search for the double mini block cipher, how many operations are needed?

      d) What is the tradeoff for the MITM attack (speed, memory, etc.)?





1 https://en.wikipedia.org/wiki/Meet-in-the-middle_attack

2 For example, https://youtu.be/S-EhbhDXUwM (It is a bit long…)

In [2]:
import random

# Define the S-box substitution table
s_box = {
    "0000": "1001", "0001": "0100", "0010": "1010", "0011": "1011",
    "0100": "1101", "0101": "0001", "0110": "1000", "0111": "0101",
    "1000": "0110", "1001": "0010", "1010": "0000", "1011": "0011",
    "1100": "1100", "1101": "1110", "1110": "1111", "1111": "0111"
}

# Define the inverse S-box for correct decryption
inv_s_box = {v: k for k, v in s_box.items()}  # Swap keys and values

# Substitution functions
def substitute(nibble):
    return s_box.get(nibble, "Invalid Input")  # Forward S-box substitution

def inverse_substitute(nibble):
    return inv_s_box.get(nibble, "Invalid Input")  # Correct inverse substitution

# Other helper functions
def shift(block):
    return block[1:] + block[0]  # Circular shift left

def inverse_shift(block):
    return block[-1] + block[:-1]  # Circular shift right (inverse)

def mix(block):
    constant = "1100"  # Example mix operation with XOR
    return ''.join(['1' if b != constant[i] else '0' for i, b in enumerate(block)])

def inverse_mix(block):
    constant = "1100"  # Same constant used in mix
    return ''.join(['1' if b != constant[i] else '0' for i, b in enumerate(block)])  # Undo mix

def add_round_key(block, key):
    return ''.join(['1' if b != k else '0' for b, k in zip(block, key)])  # XOR operation

# Encryption rounds
def encrypt_round1(block, key):
    block = substitute(block)
    block = shift(block)
    block = mix(block)
    block = add_round_key(block, key)
    return block

def encrypt_round2(block, key):
    block = substitute(block)
    block = shift(block)
    block = add_round_key(block, key)
    return block

# **Fixed decryption functions**
def decrypt_round2(block, key):
    block = add_round_key(block, key)
    block = inverse_shift(block)  # Correct inverse shift
    block = inverse_substitute(block)  # Correct inverse S-box substitution
    return block

def decrypt_round1(block, key):
    block = add_round_key(block, key)
    block = inverse_mix(block)  # Correct inverse mix function
    block = inverse_shift(block)  # Correct inverse shift
    block = inverse_substitute(block)  # Correct inverse S-box substitution
    return block

# SAES encryption & decryption
def encrypt(plaintext, keys):
    key0, key1, key2 = keys  # Unpack keys
    ciphertext = ''
    for i in range(0, len(plaintext), 4):
        block = plaintext[i:i+4]
        intermediate = encrypt_round1(block, key1)
        ciphertext += encrypt_round2(intermediate, key2)
    return ciphertext

def decrypt(ciphertext, keys):
    key0, key1, key2 = keys
    plaintext = ''
    for i in range(0, len(ciphertext), 4):
        block = ciphertext[i:i+4]
        intermediate = decrypt_round2(block, key2)
        plaintext += decrypt_round1(intermediate, key1)
    return plaintext

# Character to Binary Mapping
char_to_bin = {
    'A': '0000', 'B': '0001', 'C': '0010', 'D': '0011',
    'E': '0100', 'F': '0101', 'G': '0110', 'H': '0111',
    'I': '1000', 'J': '1001', 'K': '1010', 'L': '1011',
    'M': '1100', 'N': '1101', 'O': '1110', 'P': '1111'
}

bin_to_char = {v: k for k, v in char_to_bin.items()}  # Reverse mapping

def text_to_binary(text):
    return ''.join([char_to_bin[char] for char in text.upper() if char in char_to_bin])

def binary_to_text(binary):
    return ''.join([bin_to_char[binary[i:i+4]] for i in range(0, len(binary), 4)])

# Key Expansion & Random Key Generator
def key_expansion(key):
    return key[:4], key[4:8], key[8:12]  # Split key into three parts

def generate_random_key():
    return ''.join(random.choice('01') for _ in range(12))

# **Testing the Fix**
key = generate_random_key()
key_exp = key_expansion(key)

plaintext = "ABC"  # Example input
plaintext_binary = text_to_binary(plaintext)

print(f"Original Plaintext: {plaintext}")
print(f"Binary Plaintext: {plaintext_binary}")

ciphertext = encrypt(plaintext_binary, key_exp)
print(f"Ciphertext: {ciphertext}")

decrypted_binary = decrypt(ciphertext, key_exp)
decrypted_text = binary_to_text(decrypted_binary)

print(f"Decrypted Binary: {decrypted_binary}")
print(f"Decrypted Plaintext: {decrypted_text}")
print(f"Key: {key}")

assert plaintext == decrypted_text, "Decryption failed!"


Original Plaintext: ABC
Binary Plaintext: 000000010010
Ciphertext: 100010010010
Decrypted Binary: 000000010010
Decrypted Plaintext: ABC
Key: 011101101100


 In the pseudo code for question 2 in project 3, it says the following:
> For the project, we need to modify the pseudo code to fit our class project. The pseudo-code for this
> mini block cipher based on the SAES is as follows:
> 1. Get (Key0, Key1, Key2) from Key K using the key expansion


It says to get key0, key1, and key2

But then steps 2 and 3 say to use key1 and key2, completely skipping key0

The pseudo-code for SAES says to use key0, before either of the encrypt rounds. Meaning key0 actually gets used.

But I don't see where key0 gets used in the mini block cipherSo that's why I'm messaging you.Why does key0 get made if it's not being used?

Secondly. For the substitute portion of the algorithm are we using the substitution box from the slides of this week's lecture?



## Task 1A

In [13]:
import random

# Define the S-box substitution table (adjusted for 5-bit input)
s_box = {
    "00000": "11010", "00001": "10011", "00010": "01000", "00011": "10100",
    "00100": "01101", "00101": "11110", "00110": "00111", "00111": "10101",
    "01000": "00010", "01001": "11001", "01010": "01110", "01011": "11100",
    "01100": "10000", "01101": "00110", "01110": "01011", "01111": "00001",
    "10000": "10110", "10001": "10001", "10010": "00011", "10011": "11011",
    "10100": "00101", "10101": "11111", "10110": "01001", "10111": "10010",
    "11000": "01111", "11001": "00000", "11010": "11000", "11011": "10111",
    "11100": "00100", "11101": "01100", "11110": "01010", "11111": "11101"
}

inv_s_box = {v: k for k, v in s_box.items()}  # Reverse mapping

# Substitution functions
def substitute(nibble):
    if nibble not in s_box:
        raise ValueError(f"Invalid S-Box input: {nibble}")
    return s_box[nibble]

def inverse_substitute(nibble):
    if nibble not in inv_s_box:
        raise ValueError(f"Invalid Inverse S-Box input: {nibble}")
    return inv_s_box[nibble]


def shift(block):
    return block[2:] + block[:2]  # Shift by 2

def inverse_shift(block):
    return block[-2:] + block[:-2]  # Reverse shift by 2


def mix(block):
    constant = "11010"  # block size 5 bits
    return ''.join(['1' if b != constant[i] else '0' for i, b in enumerate(block)])

def inverse_mix(block):
    constant = "11010"  # Ensure block size consistency
    return ''.join(['1' if b != constant[i] else '0' for i, b in enumerate(block)])

def add_round_key(block, key):
    return ''.join(['1' if b != k else '0' for b, k in zip(block, key)])  # XOR

# Encryption rounds (updated for 5-bit blocks)
def encrypt_round1(block, key):
    block = substitute(block)
    block = shift(block)
    block = mix(block)
    block = add_round_key(block, key)
    return block

def encrypt_round2(block, key):
    block = substitute(block)
    block = shift(block)
    block = add_round_key(block, key)
    return block

# **Decryption functions (updated for 5-bit blocks)**
def decrypt_round2(block, key):
    block = add_round_key(block, key)
    block = inverse_shift(block)
    block = inverse_substitute(block)
    return block

def decrypt_round1(block, key):
    block = add_round_key(block, key)
    block = inverse_mix(block)
    block = inverse_shift(block)
    block = inverse_substitute(block)
    return block

# **SAES Encryption & Decryption**
def encrypt(plaintext, keys):
    key0, key1, key2 = keys  # Unpack keys
    ciphertext = ''
    for i in range(0, len(plaintext), 5):  # Process full 5-bit blocks
        block = plaintext[i:i+5]
        if len(block) < 5:
            continue  # Ignore incomplete blocks (optional)
        intermediate = encrypt_round1(block, key1)
        ciphertext += encrypt_round2(intermediate, key2)
    return ciphertext

def decrypt(ciphertext, keys):
    key0, key1, key2 = keys
    plaintext = ''
    for i in range(0, len(ciphertext), 5):  # Process full 5-bit blocks
        block = ciphertext[i:i+5]
        if len(block) < 5:
            continue  # Ignore incomplete blocks (optional)
        intermediate = decrypt_round2(block, key2)
        plaintext += decrypt_round1(intermediate, key1)
    return plaintext

# **Character to Binary Mapping (5-bit)**
char_to_bin = {
    'A': '00000', 'B': '00001', 'C': '00010', 'D': '00011',
    'E': '00100', 'F': '00101', 'G': '00110', 'H': '00111',
    'I': '01000', 'J': '01001', 'K': '01010', 'L': '01011',
    'M': '01100', 'N': '01101', 'O': '01110', 'P': '01111',
    'Q': '10000', 'R': '10001', 'S': '10010', 'T': '10011',
    'U': '10100', 'V': '10101', 'W': '10110', 'X': '10111',
    'Y': '11000', 'Z': '11001', ' ': '11010'
}

bin_to_char = {v: k for k, v in char_to_bin.items()}  # Reverse mapping

def text_to_binary(text):
    return ''.join([char_to_bin[char] for char in text.upper() if char in char_to_bin])

def binary_to_text(binary):
    return ''.join([bin_to_char[binary[i:i+5]] for i in range(0, len(binary), 5)])

# **Key Expansion & Random Key Generator (16-bit)**
def key_expansion(key):
    return key[:6], key[6:11], key[11:16]  # 6-bit Key0, 5-bit Key1, 5-bit Key2

def generate_random_key():
    return ''.join(random.choice('01') for _ in range(16))

# **Testing the Fix**
key = generate_random_key()
key_exp = key_expansion(key)

plaintext = "HELLO TEST WORLD"  # Example input
plaintext_binary = text_to_binary(plaintext)

ciphertext = encrypt(plaintext_binary, key_exp)
decrypted_binary = decrypt(ciphertext, key_exp)
decrypted_text = binary_to_text(decrypted_binary)

print(f"Original Plaintext: {plaintext}")
print(f"Binary Plaintext: {plaintext_binary}")
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Binary: {decrypted_binary}")
print(f"Decrypted Plaintext: {decrypted_text}")
print(f"Key: {key}")

assert plaintext == decrypted_text, "Decryption failed!"


Original Plaintext: HELLO TEST WORLD
Binary Plaintext: 00111001000101101011011101101010011001001001010011110101011001110100010101100011
Ciphertext: 00011001010011000110010110000111110001010010011110000011011001011100010011001001
Decrypted Binary: 00111001000101101011011101101010011001001001010011110101011001110100010101100011
Decrypted Plaintext: HELLO TEST WORLD
Key: 1001110000000001


In [2]:
import random
import string

# Define the S-box substitution table (adjusted for 8-bit input)
# Define the S-Box
s_box = {}

# Populate the S-Box
for i in range(256):
    bin_i = f"{i:08b}"  # Convert i to 8-bit binary string
    bin_val = f"{(i * 3) % 256:08b}"  # Calculate the S-Box value
    s_box[bin_i] = bin_val

# Define the Inverse S-Box
inv_s_box = {}

# Populate the Inverse S-Box
for key, value in s_box.items():
    inv_s_box[value] = key  # Reverse mapping for the Inverse S-Box

# Substitution functions
def substitute(byte):
    if byte not in s_box:
        raise ValueError(f"Invalid S-Box input: {byte}")
    return s_box[byte]

def inverse_substitute(byte):
    if byte not in inv_s_box:
        raise ValueError(f"Invalid Inverse S-Box input: {byte}")
    return inv_s_box[byte]

def shift(block):
    return block[1:] + block[:1]  # Shift by 1 (8-bit block)

def inverse_shift(block):
    return block[-1:] + block[:-1]  # Reverse shift by 1

def mix(block):
    constant = "10101010"  # block size 8 bits
    return ''.join(['1' if b != constant[i] else '0' for i, b in enumerate(block)])

def inverse_mix(block):
    constant = "10101010"  # Ensure block size consistency
    return ''.join(['1' if b != constant[i] else '0' for i, b in enumerate(block)])

def add_round_key(block, key):
    return ''.join(['1' if b != k else '0' for b, k in zip(block, key)])  # XOR

# Encryption rounds (updated for 8-bit keys and 8-bit blocks)
def encrypt_round1(block, key):
    block = substitute(block)
    block = shift(block)
    block = mix(block)
    block = add_round_key(block, key)
    return block

def encrypt_round2(block, key):
    block = substitute(block)
    block = shift(block)
    block = add_round_key(block, key)
    return block

# **Decryption functions (updated for 8-bit blocks and 16-bit keys)**
def decrypt_round2(block, key):
    block = add_round_key(block, key)
    block = inverse_shift(block)
    block = inverse_substitute(block)
    return block

def decrypt_round1(block, key):
    block = add_round_key(block, key)
    block = inverse_mix(block)
    block = inverse_shift(block)
    block = inverse_substitute(block)
    return block

# **SAES Encryption & Decryption**
def encrypt(plaintext, keys):
    key0, key1, key2 = keys  # Unpack keys
    ciphertext = ''
    for i in range(0, len(plaintext), 8):  # Process full 8-bit blocks
        block = plaintext[i:i+8]
        if len(block) < 8:
            continue  # Ignore incomplete blocks (optional)
        intermediate = encrypt_round1(block, key1)
        ciphertext += encrypt_round2(intermediate, key2)
    return ciphertext

def decrypt(ciphertext, keys):
    key0, key1, key2 = keys
    plaintext = ''
    for i in range(0, len(ciphertext), 8):  # Process full 8-bit blocks
        block = ciphertext[i:i+8]
        if len(block) < 8:
            continue  # Ignore incomplete blocks (optional)
        intermediate = decrypt_round2(block, key2)
        plaintext += decrypt_round1(intermediate, key1)
    return plaintext

# **Character to Binary Mapping (8-bit)**
# Using ASCII printable characters (256 characters)
char_to_bin = {ch: f"{ord(ch):08b}" for ch in string.printable}

# Reverse mapping
bin_to_char = {v: k for k, v in char_to_bin.items()}

def text_to_binary(text):
    return ''.join([char_to_bin[char] for char in text if char in char_to_bin])

def binary_to_text(binary):
    # Handle incomplete blocks if needed (padded with extra 0's)
    return ''.join([bin_to_char[binary[i:i+8]] for i in range(0, len(binary), 8)])

# **Key Expansion & Random Key Generator (16-bit for key0, key1, key2)**
def key_expansion(key):
    key0 = key[:16]   # First 16 bits for key0
    key1 = key[16:32] # Next 16 bits for key1
    key2 = key[32:48] # Last 16 bits for key2
    return key0, key1, key2  # Return the 3 keys

def generate_random_key():
    return ''.join(random.choice('01') for _ in range(48))  # 48-bit key

# **Testing the Fix**
key = generate_random_key()
key_exp = key_expansion(key)

plaintext = "HELLO, World!"  # Example input with all printable characters
plaintext_binary = text_to_binary(plaintext)

ciphertext = encrypt(plaintext_binary, key_exp)
decrypted_binary = decrypt(ciphertext, key_exp)
decrypted_text = binary_to_text(decrypted_binary)

print(f"Original Plaintext: {plaintext}")
print(f"Binary Plaintext: {plaintext_binary}")
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Binary: {decrypted_binary}")
print(f"Decrypted Plaintext: {decrypted_text}")
print(f"Key: {key}")

assert plaintext == decrypted_text, "Decryption failed!"


Original Plaintext: HELLO, World!
Binary Plaintext: 01001000010001010100110001001100010011110010110000100000010101110110111101110010011011000110010000100001
Ciphertext: 01001111010100101001111110011111110010110001111111101001000101011011010111100001000110000011100010001101
Decrypted Binary: 01001000010001010100110001001100010011110010110000100000010101110110111101110010011011000110010000100001
Decrypted Plaintext: HELLO, World!
Key: 011000111011100000011010101100110100100100100011


The above code is likely the final version of Task 1a. Unless someone thinks that we should add lower case letters/numbers/special chars. Which could be acheived with slight modification. But I'm not sure it's needed.

Why I think it's the final version:


*   16 bit key
*   Full alphabet
*   Spaces allowed
*   Works without issue





## Task 1B

In [3]:
# Make at least ten pairs of plaintexts and ciphertexts.

pt_1 = "HELLO"
pt_2 = "WORLD"
pt_3 = "GROUP SEVEN"
pt_4 = "IS THE GREATEST GROUP IN CSGY 6903 HISTORY"
pt_5 = "SEAN"
pt_6 = "XIJIANG"
pt_7 = "ELIZAVETA"
pt_8 = "LEI"
pt_9 = "CRYPTOGRAPHY"
pt_10 = "IF TWO CRYPTOGRAPHERS WALK INTO A BAR NOBODY ELSE HAS A CLUE WHAT THEY'RE TALKING ABOUT"


ct = {pt_1 : encrypt( text_to_binary(pt_1), key_exp),
pt_2 : encrypt( text_to_binary(pt_2), key_exp),
pt_3 : encrypt( text_to_binary(pt_3), key_exp),
pt_4 : encrypt( text_to_binary(pt_4), key_exp),
pt_5 : encrypt( text_to_binary(pt_5), key_exp),
pt_6 : encrypt( text_to_binary(pt_6), key_exp),
pt_7 : encrypt( text_to_binary(pt_7), key_exp),
pt_8 : encrypt( text_to_binary(pt_8), key_exp),
pt_9 : encrypt( text_to_binary(pt_9), key_exp),
pt_10 : encrypt( text_to_binary(pt_10), key_exp)
      }

print(ct)


{'HELLO': '0100111101010010100111111001111111001011', 'WORLD': '0001010111001011011001111001111110111111', 'GROUP SEVEN': '1110101101100111110010111001001010101110111010011101101001010010011100010101001000010111', 'IS THE GREATEST GROUP IN CSGY 6903 HISTORY': '011000111101101011101001111111100100111101010010111010011110101101100111010100100000001011111110010100101101101011111110111010011110101101100111110010111001001010101110111010010110001100010111111010011001101111011010111010111010110011101001111101100010001000101110010110101110100101001111011000111101101011111110110010110110011110101100', 'SEAN': '11011010010100100000001000010111', 'XIJIANG': '10001000011000110000011101100011000000100001011111101011', 'ELIZAVETA': '010100101001111101100011010000010000001001110001010100101111111000000010', 'LEI': '100111110101001001100011', 'CRYPTOGRAPHY': '100110110110011110101100101011101111111011001011111010110110011100000010101011100100111110101100', "IF TWO CRYPTOGRAPHERS WALK INTO A BAR NOBODY

## Task 2A

In [16]:
def mitm_attack(plaintext, ciphertext, key_size=16):
    encrypt_results = {}

    # Step 1: Store encrypted values for possible key1 matches
    for key1 in range(2**key_size):
        key1_bin = f"{key1:016b}"

        # Ensure processing of 8-bit blocks
        intermediate_values = []
        for i in range(0, len(plaintext), 8):
            block = plaintext[i:i+8]
            if len(block) < 8:
                continue  # Ignore incomplete blocks
            X = encrypt_round1(block, key1_bin)
            intermediate_values.append(X)

        encrypt_results[tuple(intermediate_values)] = key1_bin

    print("Encryption phase done. Now checking decryption phase for matches...")

    # Step 2: Compute X' = decrypt_round2(Key2, C) and find matches in encrypt_results
    for key2 in range(2**key_size):
        key2_bin = f"{key2:016b}"

        decrypted_values = []
        for i in range(0, len(ciphertext), 8):
            block = ciphertext[i:i+8]
            if len(block) < 8:
                continue
            X_prime = decrypt_round2(block, key2_bin)
            decrypted_values.append(X_prime)

        if tuple(decrypted_values) in encrypt_results:
            key1_bin = encrypt_results[tuple(decrypted_values)]
            print("\nMatching Key Pair Found:")
            print(f"Key1: {key1_bin}, Key2: {key2_bin}")

            return key1_bin, key2_bin  # Return the first found match

    print("\nNo matching key pair found.")
    return None, None

## Task 2B

In [19]:
# Perform MITM attack with a known plaintext-ciphertext pair
import time

plaintext_binary = text_to_binary("HELLO, TEST World!")  # Convert known plaintext to binary
ciphertext = encrypt(plaintext_binary, key_exp)  # Encrypt plaintext using the real keys

start_time = time.time()
key1_found, key2_found = mitm_attack(plaintext_binary, ciphertext)
end_time = time.time()

# **Validate the found keys**
if key1_found and key2_found:
    recovered_plaintext = decrypt(ciphertext, ("", key1_found, key2_found))
    recovered_text = binary_to_text(recovered_plaintext)

    print(f"\nRecovered Plaintext: {recovered_text}")
    print(f"\nMITM Attack Time: {end_time - start_time} seconds")

    # **Check if attack was successful**
    if recovered_text == "HELLO, TEST World!":
        print("\nMITM Attack Successful! The keys are correct.")
    else:
        print("\nMITM Attack Failed. The recovered plaintext is incorrect.")
else:
    print("\nMITM Attack Failed. No valid key pair found.")

Encryption phase done. Now checking decryption phase for matches...

Matching Key Pair Found:
Key1: 1001101011111111, Key2: 0100100000000000

Recovered Plaintext: HELLO, TEST World!

MITM Attack Time: 3.847144603729248 seconds

MITM Attack Successful! The keys are correct.
